In [ ]:
pip install -r requirements.txt

In [1]:
import pandas as pd
from siuba import _, select, rename, mutate, group_by, arrange, filter, summarize, distinct, left_join

In [2]:
census = pd.read_csv('cleaned_data/census_demographics.csv')
health = pd.read_csv('cleaned_data/asthma_clean_file.csv')
#wildfires = pd.read_csv('.csv')
#air_quality = pd.read_csv('.csv')

In [3]:
census = (census
  >> mutate(
      White_Perc = _.TotalWhiteNotHispanic/_.TotalPopulation,
      Poverty_Perc = _.TotalBelowPovertyLine/_.TotalPopulation,
      Total_Uninsured = _.TotalEmployedNoInsurance + _.TotalUnemployedNoInsurance + _.TotalNotInLaborForceNoInsurance,
      Uninsured_Perc = _.Total_Uninsured/_.TotalPopulation
      )
  )

In [4]:
df_merge = (health
  >> filter(_.Category == "Total Population", _.Age == "18 and Over")
  >> rename(County = "Geography")
  >> left_join(_, select(census, _.County, _.TotalPopulation, _.MedianIncome, _.White_Perc, _.Poverty_Perc, _.Uninsured_Perc), on = "County")
  >> left_join(_, wildfires, on = ["County","Year"])
  >> left_join(_, air_quality, on = ["County","Year"])
  >> select(_.County, _.Year, _.Total_Population == _.TotalPopulation, _.Median_Income == _.MedianIncome, _.White_Perc, _.Poverty_Perc, _.Uninsured_Perc,
            _.Asthma_Incidents == _.Count, _.Asthma_Rate == _.Rate, )
  >> arrange(_.County, _.Year)
)

In [5]:
df_merge

,County,Year,Total_Population,Median_Income,White_Perc,Poverty_Perc,Uninsured_Perc,Asthma_Incidents,Asthma_Rate
224,Alameda,2011,1656754.0,43583.0,0.314137,0.097529,0.037430,6706,56.6
168,Alameda,2012,1656754.0,43583.0,0.314137,0.097529,0.037430,6965,58.4
112,Alameda,2013,1656754.0,43583.0,0.314137,0.097529,0.037430,6705,55.0
56,Alameda,2014,1656754.0,43583.0,0.314137,0.097529,0.037430,6332,51.6
0,Alameda,2015,1656754.0,43583.0,0.314137,0.097529,0.037430,6513,53.1
...,...,...,...,...,...,...,...,...,...
55,Yuba,2015,76360.0,27877.0,0.551925,0.151545,0.060071,215,39.5
336,Yuba,2016,76360.0,27877.0,0.551925,0.151545,0.060071,202,37.7
392,Yuba,2017,76360.0,27877.0,0.551925,0.151545,0.060071,245,45.1
448,Yuba,2018,76360.0,27877.0,0.551925,0.151545,0.060071,227,40.8


In [ ]:
df_merge.to_csv('cleaned_data/complete_merged_table.csv', encoding = 'utf-8', index = False)